# Prueba 5

In [1]:
import torch
import torch.nn as nn
from pytorch_utils import *

%matplotlib notebook
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

## Training CNN from stratch

Vamos a hacer una red CNN desde cero (fácil) y la vamos a entrenar con la base de datos CIFAR10. Después vamos a querer hacer finetunning de una red preentrenada. 

In [2]:
NUM_TRAIN = 49000
NUM_VAL = 1000
batch_size = 64

train_dataloader, val_dataloader, test_dataloader = GetCIFAR10DataLoaders(NUM_TRAIN, 
                                                                          NUM_VAL, 
                                                                          batch_size)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

La base de datos CIFAR10 contiene 60000 muestras, cada una constituída por una imagen y la clase a la que pertenece.
Cada muestra contiene dos componentes:
1) Input vector: imagen de tamaño 3x224x224 con valores normalizados manualmente
2) Label: ínidice de la lista de clases posibles: 
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Cantidad de muestras para entrenamiento:  49000
Cantidad de muestras para validación:  1000
Cantidad de muestras para test:  10000


In [6]:
class CNNFromStratch(nn.Module):
    
    def __init__(self):
        super(CNNFromStratch, self).__init__()
        in_features = (3, 32, 32)
        out_features = 10
        
        # Layers del forward pass:
        kernel_size = (6,6)
        stride = (2,2)
        padding = (0,0)
        channels = 64
        self.conv = nn.Conv2d(3, channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.act1 = nn.ReLU()
        self.linear = nn.Linear(((in_features[1] + padding[0] - kernel_size[0])//stride[0] + 1) * \
                                ((in_features[2] + padding[1] - kernel_size[1])//stride[1] + 1) * channels,
                                out_features)
        
        # Función loss:
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self, x):
        #N_batch, C, H, W = x.size()
        #return self.linear(self.act1(self.conv(x)).view(N_batch, -1))
        
    

# Instancia del modelo:
model = CNNFromStratch()

In [7]:
# Especificaciones de cómo adquirir los datos para entrenamiento:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
data = {
    'device': device,
    'input_dtype': torch.float32, 
    'target_dtype': torch.long,
    'train_dataloader': train_dataloader,
    'val_dataloader': val_dataloader
}

# Hiperparámetros del modelo y otros:
epochs = 1 # Cantidad de epochs
sample_loss_every = 100 # Cantidad de iteraciones para calcular la cantidad de aciertos
learning_rate = 1e-2 # Tasa de aprendizaje

# Entrenamiento:
performance_history = train(model, data, epochs, learning_rate, sample_loss_every)

RuntimeError: size mismatch, m1: [64 x 774400], m2: [12544 x 10] at /opt/conda/conda-bld/pytorch_1570910687650/work/aten/src/THC/generic/THCTensorMathBlas.cu:290

In [14]:
performance_history['loss']

[2.3535523414611816,
 1.7592602968215942,
 1.692118763923645,
 1.5023483037948608,
 1.755894660949707,
 1.4654757976531982,
 1.3111964464187622,
 1.5826289653778076]

# Finetuning AlexNet

Vamos a tratar de usar algunos layers Preentrenados de AlexNet. 

In [2]:
NUM_TRAIN = 49000
NUM_VAL = 1000
batch_size = 64

train_dataloader, val_dataloader, test_dataloader = GetCIFAR10DataLoaders(NUM_TRAIN, 
                                                                          NUM_VAL, 
                                                                          batch_size)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

La base de datos CIFAR10 contiene 60000 muestras, cada una constituída por una imagen y la clase a la que pertenece.
Cada muestra contiene dos componentes:
1) Input vector: imagen de tamaño 3x224x224 con valores normalizados manualmente
2) Label: ínidice de la lista de clases posibles: 
['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Cantidad de muestras para entrenamiento:  49000
Cantidad de muestras para validación:  1000
Cantidad de muestras para test:  10000


In [3]:
import torchvision.models as models

In [7]:
class MyAlexNet(nn.Module):
    
    def __init__(self, feature_extracting=True):
        super(MyAlexNet, self).__init__()
        
        # Layers para el forward pass:
        alexnet = models.alexnet(pretrained=True, progress=True)
        self.features = alexnet.features
        if feature_extracting:
            for param in self.features.parameters():
                param.requires_grad = False
        self.fc = nn.Linear(256 * 6 * 6, 10)
        
        # Función de loss:
        self.loss = nn.CrossEntropyLoss()
        
    def forward(self,x):
        N, C, H, W = x.size()
        return self.fc(self.features(x).view(N,-1))
        
        
model = MyAlexNet(feature_extracting=False)

In [8]:
# Especificaciones de cómo adquirir los datos para entrenamiento:

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

data = {
    'device': device,
    'input_dtype': torch.float32, 
    'target_dtype': torch.long,
    'train_dataloader': train_dataloader,
    'val_dataloader': val_dataloader
}

# Hiperparámetros del modelo y otros:
epochs = 2 # Cantidad de epochs
sample_loss_every = 50 # Cantidad de iteraciones para calcular la cantidad de aciertos
learning_rate = 1e-2 # Tasa de aprendizaje

# Entrenamiento:
performance_history = train(model, data, epochs, learning_rate, sample_loss_every)

Epoch: 0, Iteration: 0, Accuracy: 153/1000 
Epoch: 0, Iteration: 50, Accuracy: 667/1000 
Epoch: 0, Iteration: 100, Accuracy: 757/1000 
Epoch: 0, Iteration: 150, Accuracy: 786/1000 
Epoch: 0, Iteration: 200, Accuracy: 757/1000 
Epoch: 0, Iteration: 250, Accuracy: 813/1000 
Epoch: 0, Iteration: 300, Accuracy: 823/1000 
Epoch: 0, Iteration: 350, Accuracy: 804/1000 
Epoch: 0, Iteration: 400, Accuracy: 783/1000 
Epoch: 0, Iteration: 450, Accuracy: 834/1000 
Epoch: 0, Iteration: 500, Accuracy: 851/1000 
Epoch: 0, Iteration: 550, Accuracy: 801/1000 
Epoch: 0, Iteration: 600, Accuracy: 840/1000 
Epoch: 0, Iteration: 650, Accuracy: 814/1000 
Epoch: 0, Iteration: 700, Accuracy: 851/1000 
Epoch: 0, Iteration: 750, Accuracy: 838/1000 
Epoch: 1, Iteration: 0, Accuracy: 844/1000 
Epoch: 1, Iteration: 50, Accuracy: 843/1000 
Epoch: 1, Iteration: 100, Accuracy: 844/1000 
Epoch: 1, Iteration: 150, Accuracy: 857/1000 
Epoch: 1, Iteration: 200, Accuracy: 867/1000 
Epoch: 1, Iteration: 250, Accuracy: 856/